In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml==1.1.6

In [1]:
import pandas as pd

In [2]:
# URL of the CSV file on GitHub
url = 'https://github.com/aksh301091/fdc_akshaya_git/blob/91a0b3faf99492355d3816919fbf79de434926c3/ME_CASINO_PRJ/customer_table.csv'
#'https://raw.githubusercontent.com/username/repository/branch/filename.csv'

In [5]:
url

'https://github.com/aksh301091/fdc_akshaya_git/blob/91a0b3faf99492355d3816919fbf79de434926c3/ME_CASINO_PRJ/customer_table.csv'

In [9]:
# Read the CSV file into a DataFrame
df = pd.read_csv('customer_table.csv')

# Display the first few rows of the DataFrame
print(df.head())

   PLAYER_ID  AGE  GENDER HOME_COUNTRY   HOME_CITY DATE_FIRST_VISIT  \
0          1   28  Female        India   Hyderabad       02-10-2021   
1          2   69  Female    Singapore   Singapore       06-09-2021   
2          3   52  Female    Singapore   Singapore       27-11-2021   
3          4   41  Female        India       Delhi       10-10-2021   
4          5   66  Female           UK  Manchester       08-09-2021   

  DATE_LAST_VISIT  TOTAL_NUMBER_OF_VISITS  TOTAL_DURATION_SPENT  \
0      19-08-2024                      24                111.17   
1      17-08-2024                      24                114.26   
2      22-06-2024                      17                 81.23   
3      18-05-2024                      17                 69.72   
4      01-08-2024                      15                 70.89   

   AVERAGE_DURATION_PER_VISIT  ...  TOTAL_AMOUNT_SPENT_IN_HOTEL  \
0                    4.632083  ...                     26043.68   
1                    4.760833  ...  